In [12]:
from tkinter import Tk, Label, Button, TOP
from tkinter import filedialog
import numpy as np
from PIL import ImageTk, Image


def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return dice_coef(y_true, y_pred)
model = keras.models.load_model("model.h5",custom_objects={'dice_coef_loss': dice_coef_loss,'dice_coef':dice_coef})

def classify_image():
    global photo, input_label, output_label
    img_rows = 96
    img_cols = 96

    file_path = filedialog.askopenfilename()
    
    image = Image.open(file_path)
    
    img = image.resize((img_rows, img_cols))
    img = img.convert('L')
    
    img_array = np.array(img) / 255.0
    img_array = img_array.reshape((1, img_rows, img_cols, 1))
    
    prediction = model.predict(img_array)
    
    prediction_2d = np.squeeze(prediction, axis=(0, 3))
    prediction_2d = (prediction_2d - prediction_2d.min()) / (prediction_2d.max() - prediction_2d.min()) * 255.0
    prediction_image = Image.fromarray(prediction_2d.astype(np.uint8), mode='L')
    
    input_photo = ImageTk.PhotoImage(image)
    output_photo = ImageTk.PhotoImage(prediction_image)
    
    input_label.config(image=input_photo)
    output_label.config(image=output_photo)
    
    input_label.image = input_photo
    output_label.image = output_photo


root = Tk()

root.title("Nerve Segmentor")
root.configure(background="black")

heading = Label(root, text="Nerve Segmentor", font=("Arial", 24, "bold"), bg="black", fg="white")
heading.pack(pady=20)

input_label = Label(root, text="Input Image",  font=("Helvetica", 10), bg="black", fg="white")
input_label.pack(side=TOP, pady=20)

output_label = Label(root, text="Output Image", font=("Helvetica", 10), bg="black", fg="white")
output_label.pack(side=TOP, pady=20)

button = Button(root, text="Select Image", bg="white", fg="black", font=("Helvetica", 10), command=classify_image)
button.pack(side=TOP, pady=20)

root.mainloop()
